In [1]:
import numpy as np
import pandas as pd
import sys, os
from collections import defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../Shared')
import processing, Model, Visualization


def call_data(base_path, sub_lst, sub_idx):
    data = pd.read_csv(base_path+sub_lst[sub_idx])
    return data

def data_setup(lst_normal, path_normal, window_size, step_size):
    subjects_data = defaultdict(list)
    subjects_label = defaultdict(list)

    for s_idx, filename in enumerate(lst_normal):
        name = filename.lower()
        subject_id = name.split("n")[0]
        data = call_data(path_normal, lst_normal, s_idx).to_numpy()  # (num_samples, num_channels)

        windows = processing.sliding_window(data, window_size, step_size)  # (num_windows, win_len, ch)

        if "standing" in name: label = 0
        elif "gait" in name: label = 1
        elif "sitting" in name: label = 2
        else: label = -1

        labels = np.full(len(windows), label)

        subjects_data[subject_id].append(windows)
        subjects_label[subject_id].append(labels)

    return subjects_data, subjects_label



def get_X_y(X, y):
    all_X, all_y = [], []

    #for subject_id in subjects_data.keys():
    data_list = X
    label_list = y

    for data, labels in zip(data_list, label_list):
        for w, label in zip(data, labels):  # w: (win_len, ch)
            feat = processing.extract_features(w)  # (num_channels*5,)
            all_X.append(feat)
            all_y.append(label)

    all_X, all_y = np.array(all_X), np.array(all_y)   # (N, num_channels*5)

    return all_X, all_y

# Baseline performance - without adaptation

In [ ]:
path_abnormal = "../../../Data/Gait1/Abnormal/"
path_normal = "../../../Data/Gait1/normal/"

lst_abnormal = os.listdir(path_abnormal)
lst_normal = os.listdir(path_normal)

window_size, step_size = 200, 10
num_channels, num_features = 5, 5  # we extracted 5 features per channel

subjects_data, subjects_label = data_setup(lst_normal, path_normal, window_size, step_size)
ACC_all = []

for subject_id in subjects_data.keys():
    print(f"Sub ID {subject_id}")
    X, y = get_X_y(X=subjects_data[subject_id], y=subjects_label[subject_id])
    X = X.reshape(-1, num_channels, num_features, 1)

    model = Model.build_model(input_shape=X.shape[1:], num_classes=len(np.unique(y)))
    history = model.fit(X, y, epochs=50, batch_size=64, verbose=0)
    Sub_acc = []

    for subject_id_target in subjects_data.keys():
        if subject_id == subject_id_target:
            pass
        else:
            X_target, y_target = get_X_y(X=subjects_data[subject_id_target], y=subjects_label[subject_id_target])
            X_target = X_target.reshape(-1, num_channels, num_features, 1)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)

            #Visualization.learning_plot(history)
            print(f"\t Target sub {subject_id_target} acc => {acc*100:.2f}%")
            Sub_acc.append(acc)
    ACC_all.append(Sub_acc)
    print("\n")

Sub ID 10
	 Target sub 11 acc => 58.09%
	 Target sub 1 acc => 90.92%
	 Target sub 2 acc => 90.91%
	 Target sub 3 acc => 68.91%
	 Target sub 4 acc => 62.77%
	 Target sub 5 acc => 65.09%
	 Target sub 6 acc => 77.48%
	 Target sub 7 acc => 78.83%
	 Target sub 8 acc => 79.77%
	 Target sub 9 acc => 78.59%


Sub ID 11
	 Target sub 10 acc => 79.14%
	 Target sub 1 acc => 73.26%
	 Target sub 2 acc => 72.37%
	 Target sub 3 acc => 43.86%
	 Target sub 4 acc => 46.80%
	 Target sub 5 acc => 60.43%
	 Target sub 6 acc => 51.25%
	 Target sub 7 acc => 84.73%
	 Target sub 8 acc => 59.51%
	 Target sub 9 acc => 66.56%


Sub ID 1
	 Target sub 10 acc => 82.00%
	 Target sub 11 acc => 43.43%
	 Target sub 2 acc => 98.33%
	 Target sub 3 acc => 76.33%
	 Target sub 4 acc => 63.87%
	 Target sub 5 acc => 67.94%
	 Target sub 6 acc => 90.40%
	 Target sub 7 acc => 60.78%
	 Target sub 8 acc => 50.42%
	 Target sub 9 acc => 86.80%


Sub ID 2
	 Target sub 10 acc => 83.25%
	 Target sub 11 acc => 43.52%
	 Target sub 1 acc => 

In [ ]:
ACC_all

# Few-shot adaptation